In [1]:
import pandas as pd
import numpy as np

In [10]:
df_mad_ded = pd.read_csv('../data-csv/data_madrid_ded.csv')
print('Shape:', df_mad_ded.shape)
df_mad_ded.sort_values(by=['ASSETID', 'PERIOD', 'PRICE'], ascending=[True, False, True]).head(10)

Shape: (75804, 40)


,ASSETID,PERIOD,PRICE,UNITPRICE,CONSTRUCTEDAREA,ROOMNUMBER,BATHNUMBER,HASTERRACE,HASLIFT,HASAIRCONDITIONING,...,CADMAXBUILDINGFLOOR,CADDWELLINGCOUNT,CADASTRALQUALITYID,BUILTTYPEID_1,BUILTTYPEID_2,BUILTTYPEID_3,DISTANCE_TO_CITY_CENTER,DISTANCE_TO_METRO,DISTANCE_TO_MAIN,CITY
0,1,201809,255000.0,2628.865979,97,3,2,0,1,0,...,7,8,6,0,0,1,7.829495,0.254412,3.027988,MADRID
1,2,201806,82000.0,1322.580645,62,2,1,0,1,0,...,5,11,7,0,1,0,6.435241,0.268472,4.693939,MADRID
2,3,201803,133000.0,1985.074627,67,3,1,1,0,1,...,5,17,7,0,0,1,4.836415,1.061146,2.623258,MADRID
3,4,201803,204000.0,1133.333333,180,3,2,0,1,1,...,4,13,6,0,0,1,4.664784,0.427977,3.131739,MADRID
4,5,201812,161000.0,2981.481481,54,2,1,0,0,1,...,3,7,6,0,0,1,4.063119,0.377045,2.702218,MADRID
5,6,201803,230000.0,8214.285714,28,1,1,0,1,1,...,6,13,3,0,0,1,0.241875,0.291301,1.070058,MADRID
6,7,201806,78000.0,1368.421053,57,3,1,0,0,1,...,6,25,6,0,1,0,7.269907,0.280549,4.381209,MADRID
7,8,201812,396000.0,3046.153846,130,4,2,1,1,0,...,7,87,4,0,0,1,6.453848,0.604400,5.262124,MADRID
8,9,201809,260000.0,3095.238095,84,1,1,0,1,1,...,8,98,4,0,0,1,1.560128,0.336041,1.579208,MADRID
9,10,201803,280000.0,2800.000000,100,3,2,0,1,1,...,7,203,4,0,0,1,9.142658,0.176679,8.008129,MADRID
